In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import threading
import winsound
freq=1500
dur=500
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.namedWindow ('screen', cv2.WINDOW_NORMAL)
cv2.setWindowProperty ('screen', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

def face_lines(img,x1,y1,x2,y2):
    diagonal = math.sqrt(math.pow((x1-x2),2)+math.pow((y1-y2),2))
    p=int(0.1712328*diagonal)
    cv2.line(img,(x1,y1),(x1,y1+p),(255,255,255),1)
    cv2.line(img,(x1,y2-p),(x1,y2),(255,255,255),1) 
    
    cv2.line(img,(x2,y1),(x2,y1+p),(255,255,255),1)    
    cv2.line(img,(x2,y2-p),(x2,y2),(255,255,255),1)
    
    cv2.line(img,(x1,y1),(x1+p,y1),(255,255,255),1)
    cv2.line(img,(x2-p,y1),(x2,y1),(255,255,255),1)    
    cv2.line(img,(x1,y2),(x1+p,y2),(255,255,255),1)
    cv2.line(img,(x2-p,y2),(x2,y2),(255,255,255),1)
    return img


class beeper(threading.Thread):
    def run(self):
        winsound.Beep(freq, dur)
            

cap = cv2.VideoCapture('vtest.avi')
            
frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4)) 
   
size = (frame_width, frame_height) 

human_cascade=cv2.CascadeClassifier('haarcascade_fullbody.xml')
output = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)
while(cap.isOpened()):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.putText(frame,"Maintain Social distance due to COVID-19",(40,50),font, 1.1,(255,255,255),2)
    humans=human_cascade.detectMultiScale(gray,1.05,2)
    humans_new=[]
    for (x,y,w,h) in humans:
        gray2=gray[y:y+h,x:x+w]
        humans2=human_cascade.detectMultiScale(gray2,1.03,2)
        if len(humans2)==0:
            continue
        humans_new.append((x,y,w,h))
    for j in range (len(humans_new)):
        i=humans_new[j]
        x=i[0]
        y=i[1]
        w=i[2]
        h=i[3]
        frame=face_lines(frame,x,y,x+w,y+h)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        for k in range (j+1,len(humans_new)):
            i1=humans_new[k]
            x1=i1[0]
            y1=i1[1]
            w1=i1[2]
            h1=i1[3]
            d=math.sqrt(abs((x+w/2)-(x1+w1/2))**2+abs((y+h)-(y1+h1))**2)
            d=round(d*(1/55))
            if d<2:
                beep=beeper()
                beep.start()
                cv2.line(frame,(x+int(w/2),y+h),(x1+int(w1/2),y1+h1),(0,0,255),1)
            else:
                pass
            
    output.write(frame)
    cv2.imshow('screen',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()